<a href="https://colab.research.google.com/github/pravali96/Fake-News-Classifier/blob/main/Fake-News-Classifier%20using%20LSTM%20and%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Data

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))

Saving train.csv to train (4).csv


In [ ]:
df=df.dropna()

In [ ]:
X=df.drop('label', axis=1)

In [ ]:
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences #--all sentences must have same number of words to be passed to embedded layer, gives better perf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot # convert sentence to onehot encoding given vocab_size
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
# Define vocab size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Preprocessing Data
#### Using Stemming and applying stopwords
##### Performed processing on Titles(News headlines) 

In [ ]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus[0:20]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hi

### Turning it into One-hot Representation

In [ ]:
# Each word is assigned its respective index from its word vector
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]

[4967, 1922, 1553, 155, 414, 3664, 1581, 2425, 4774, 488]

### Words Embedding - Feature representation
### Padding the sentences so that they all have equal lengths

In [ ]:
sent_length=20
# padding inorder to have all the sentences have same length of 20
embedded_docs= pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 2425, 4774,  488],
       [   0,    0,    0, ...,   80,  974, 2902],
       [   0,    0,    0, ..., 3681, 4926, 1414],
       ...,
       [   0,    0,    0, ...,  578, 3224, 1713],
       [   0,    0,    0, ..., 1546, 4046, 2675],
       [   0,    0,    0, ..., 2091, 2476, 4215]], dtype=int32)

In [ ]:
len(embedded_docs)

18285

### Creating a model
#### By adding - Embedded layer, LSTM layer and Dense layer

In [24]:
#Creating Model
# How many features or dimensions are needed?
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features , input_length=sent_length))
model.add(LSTM(100)) # one lstm with 100 neurons
model.add(Dense(1, activation='sigmoid')) # apply activation as sigmoid bcz its classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs), y.shape

(18285, (18285,))

In [27]:
import numpy as np
# Converting inputs to arrays
X_final= np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

### Creating Train-test_split

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Training the model

In [30]:
# Training 1
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 36ms/step - loss: 0.3273 - accuracy: 0.8439 - val_loss: 0.2135 - val_accuracy: 0.9125
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1357 - accuracy: 0.9460 - val_loss: 0.2034 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0936 - accuracy: 0.9661 - val_loss: 0.2357 - val_accuracy: 0.9170
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0650 - accuracy: 0.9763 - val_loss: 0.2922 - val_accuracy: 0.9112
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0512 - accuracy: 0.9836 - val_loss: 0.3308 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0308 - accuracy: 0.9904 - val_loss: 0.4144 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0213 - accuracy: 0.9933 - val_loss: 0.4541 - val_accuracy: 0.9080

In [31]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


### Performance of model 1

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3110,  309],
       [ 238, 2378]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9093620546810274

### Hyperparameter Tuning using Dropout layers

In [34]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [39]:
# Training 2
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=25,batch_size=64)

Epoch 1/25
192/192 [==============================] - 7s 35ms/step - loss: 0.0167 - accuracy: 0.9942 - val_loss: 0.5124 - val_accuracy: 0.9059
Epoch 2/25
192/192 [==============================] - 6s 34ms/step - loss: 0.0146 - accuracy: 0.9949 - val_loss: 0.5315 - val_accuracy: 0.9115
Epoch 3/25
192/192 [==============================] - 6s 34ms/step - loss: 0.0118 - accuracy: 0.9962 - val_loss: 0.5861 - val_accuracy: 0.9070
Epoch 4/25
192/192 [==============================] - 7s 34ms/step - loss: 0.0095 - accuracy: 0.9971 - val_loss: 0.6141 - val_accuracy: 0.9107
Epoch 5/25
192/192 [==============================] - 7s 34ms/step - loss: 0.0097 - accuracy: 0.9969 - val_loss: 0.5755 - val_accuracy: 0.9117
Epoch 6/25
192/192 [==============================] - 7s 34ms/step - loss: 0.0106 - accuracy: 0.9962 - val_loss: 0.5847 - val_accuracy: 0.9120
Epoch 7/25
192/192 [==============================] - 6s 34ms/step - loss: 0.0081 - accuracy: 0.9979 - val_loss: 0.5980 - val_accuracy: 0.9094

### Performance of model 2

In [40]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [41]:
confusion_matrix(y_test,y_pred)

array([[3097,  322],
       [ 216, 2400]])

In [42]:
accuracy_score(y_test,y_pred)

0.9108533554266777